# Unit 5 - Financial Planning

In [1]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import json
from MCForecastTools import MCSimulation

%matplotlib inline

In [2]:
# Load .env enviroment variables
load_dotenv() #true .env file is loaded

True

## Part 1 - Personal Finance Planner

### Collect Crypto Prices Using the `requests` Library

In [3]:
# Set current amount of crypto assets
my_btc = 1.2
my_eth = 5.3

In [4]:
# Crypto API URLs
btc_url = "https://api.alternative.me/v2/ticker/Bitcoin/?convert=CAD"
eth_url = "https://api.alternative.me/v2/ticker/Ethereum/?convert=CAD"

In [5]:
# api for btc loaded succesfully. Response = 200
btc_response = requests.get(btc_url)
btc_response


<Response [200]>

In [6]:
# api for eth loaded succesfully. Response = 200
eth_response = requests.get(eth_url)
eth_response

<Response [200]>

In [7]:
# Loaded btc_data in json and formattted it using indent = 4

btc_data = btc_response.json()
print(json.dumps( btc_data, indent=4))

{
    "data": {
        "1": {
            "id": 1,
            "name": "Bitcoin",
            "symbol": "BTC",
            "website_slug": "bitcoin",
            "rank": 1,
            "circulating_supply": 19405193,
            "total_supply": 19405193,
            "max_supply": 21000000,
            "quotes": {
                "USD": {
                    "price": 26494.0,
                    "volume_24h": 7645125270,
                    "market_cap": 514253725219,
                    "percentage_change_1h": 0.184770324504123,
                    "percentage_change_24h": 1.08908678394465,
                    "percentage_change_7d": 2.38364952097043,
                    "percent_change_1h": 0.184770324504123,
                    "percent_change_24h": 1.08908678394465,
                    "percent_change_7d": 2.38364952097043
                },
                "CAD": {
                    "price": 35992.099,
                    "volume_24h": 10385902679.295,
                    "marke

In [8]:
# Fetch current BTC price
btc_price = btc_data["data"]["1"]["quotes"]["CAD"]["price"] # retrieved BTC CAD price from the nested dictionary 
btc_price

35992.099

In [9]:
# Loaded eth_data in json and formattted it using indent = 4
eth_data = eth_response.json()
print(json.dumps( eth_data, indent=4))

{
    "data": {
        "1027": {
            "id": 1027,
            "name": "Ethereum",
            "symbol": "ETH",
            "website_slug": "ethereum",
            "rank": 2,
            "circulating_supply": 120212803,
            "total_supply": 120212803,
            "max_supply": 120212803,
            "quotes": {
                "USD": {
                    "price": 1727.33,
                    "volume_24h": 5447073607,
                    "market_cap": 207706103240,
                    "percentage_change_1h": 0.151865598884168,
                    "percentage_change_24h": 0.68549653130323,
                    "percentage_change_7d": -1.41221187098261,
                    "percent_change_1h": 0.151865598884168,
                    "percent_change_24h": 0.68549653130323,
                    "percent_change_7d": -1.41221187098261
                },
                "CAD": {
                    "price": 2346.577805,
                    "volume_24h": 7399849495.1095,
           

In [10]:
# Fetch current ETH price
eth_price = eth_data["data"]["1027"]["quotes"]["CAD"]["price"] # retrieved ETH CAD price from the nested dictionary 
eth_price


2346.577805

In [11]:
# Compute current value of my crpto
my_btc_value = my_btc * btc_price
my_eth_value = my_eth * eth_price
# Print current crypto wallet balance
print(f"The current value of your {my_btc} BTC is ${my_btc_value:0.2f}")
print(f"The current value of your {my_eth} ETH is ${my_eth_value:0.2f}")

The current value of your 1.2 BTC is $43190.52
The current value of your 5.3 ETH is $12436.86


### Collect Investments Data Using Alpaca: `SPY` (stocks) and `AGG` (bonds)

In [12]:
# Set current amount of shares
my_agg = 200
my_spy = 50

In [13]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("alpaca_api_key")
alpaca_secret_key = os.getenv("alpaca_secret_key")

 # Verifying that Alpaca key and secret were correctly loaded
print(f"Alpaca Key type: {type(alpaca_api_key)}")
print(f"Alpaca Secret Key type: {type(alpaca_secret_key)}")


Alpaca Key type: <class 'str'>
Alpaca Secret Key type: <class 'str'>


In [14]:
# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [15]:
# Format current date as ISO format
current_date = pd.Timestamp("2023-06-13", tz="America/New_York").isoformat()

# Set the tickers
tickers = ["AGG", "SPY"]

# Set timeframe to "1Day" for Alpaca API
timeframe = "1Day"



In [16]:
# Get current closing prices for SPY and AGG
portfolio = alpaca.get_bars(
    tickers,
    timeframe,
    start = current_date,
    end = current_date
).df

portfolio



,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2023-06-13 04:00:00+00:00,98.48,98.48,97.6001,97.68,9568596,27973,97.819261,AGG
2023-06-13 04:00:00+00:00,435.32,437.33,434.6300,436.66,95912772,673273,436.003879,SPY


In [18]:
# Reorganize the DataFrame

# Separate ticker data
agg = portfolio[portfolio['symbol']=='AGG'].drop('symbol', axis=1)
spy = portfolio[portfolio['symbol']=='SPY'].drop('symbol', axis=1)


# Concatenate the ticker DataFrames
multi_index_portfolio = pd.concat([agg,spy],axis=1, keys=["AGG","SPY"])

# Preview DataFrame
multi_index_portfolio.head()


AGG                                              \
                            open   high      low  close   volume trade_count   
timestamp                                                                      
2023-06-13 04:00:00+00:00  98.48  98.48  97.6001  97.68  9568596       27973   

                                         SPY                          \
                                vwap    open    high     low   close   
timestamp                                                              
2023-06-13 04:00:00+00:00  97.819261  435.32  437.33  434.63  436.66   

                                                             
                             volume trade_count        vwap  
timestamp                                                    
2023-06-13 04:00:00+00:00  95912772      673273  436.003879

In [21]:
# Pick AGG and SPY close prices
agg_close_price = float(multi_index_portfolio["AGG"]["close"])
spy_close_price = float(multi_index_portfolio["SPY"]["close"])


# Print AGG and SPY close prices
print(f"Current AGG closing price: ${agg_close_price}")
print(f"Current SPY closing price: ${spy_close_price}")




Current AGG closing price: $97.68
Current SPY closing price: $436.66


In [22]:
# Compute the current value of shares
my_agg_value = my_agg * agg_close_price
my_spy_value = my_spy * spy_close_price

# Print current value of shares
print(f"The current value of your {my_spy} SPY shares is ${my_spy_value:0.2f}")
print(f"The current value of your {my_agg} AGG shares is ${my_agg_value:0.2f}")

The current value of your 50 SPY shares is $21833.00
The current value of your 200 AGG shares is $19536.00


### Savings Health Analysis

In [ ]:
# Set monthly household income
# YOUR CODE HERE!

# Consolidate financial assets data
# YOUR CODE HERE!

# Create savings DataFrame
# YOUR CODE HERE!

# Display savings DataFrame
display(df_savings)

In [ ]:
# Plot savings pie chart
# YOUR CODE HERE!

In [ ]:
# Set ideal emergency fund
emergency_fund = monthly_income * 3

# Calculate total amount of savings
# YOUR CODE HERE!

# Validate saving health
# YOUR CODE HERE!

## Part 2 - Retirement Planning

### Monte Carlo Simulation

In [ ]:
# Set start and end dates of five years back from today.
# Sample results may vary from the solution based on the time frame chosen
start_date = pd.Timestamp('2016-05-01', tz='America/New_York').isoformat()
end_date = pd.Timestamp('2021-05-01', tz='America/New_York').isoformat()

In [ ]:
# Get 5 years' worth of historical data for SPY and AGG
# YOUR CODE HERE!

# Reorganize the DataFrame
# Separate ticker data
# YOUR CODE HERE!


# Concatenate the ticker DataFrames
# YOUR CODE HERE!

# Display sample data
df_stock_data.head()

In [ ]:
# Configuring a Monte Carlo simulation to forecast 30 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Printing the simulation input data
# YOUR CODE HERE!

In [ ]:
# Running a Monte Carlo simulation to forecast 30 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Plot simulation outcomes
# YOUR CODE HERE!

In [ ]:
# Plot probability distribution and confidence intervals
# YOUR CODE HERE!

### Retirement Analysis

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
# YOUR CODE HERE!

# Print summary statistics
# YOUR CODE HERE!

### Calculate the expected portfolio return at the `95%` lower and upper confidence intervals based on a `$20,000` initial investment.

In [ ]:
# Set initial investment
initial_investment = 20000

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $20,000
# YOUR CODE HERE!

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower} and ${ci_upper}")

### Calculate the expected portfolio return at the `95%` lower and upper confidence intervals based on a `50%` increase in the initial investment.

In [ ]:
# Set initial investment
initial_investment = 20000 * 1.5

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $30,000
# YOUR CODE HERE!

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 30 years will end within in the range of"
      f" ${ci_lower} and ${ci_upper}")

## Optional Challenge - Early Retirement


### Five Years Retirement Option

In [ ]:
# Configuring a Monte Carlo simulation to forecast 5 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Running a Monte Carlo simulation to forecast 5 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Plot simulation outcomes
# YOUR CODE HERE!

In [ ]:
# Plot probability distribution and confidence intervals
# YOUR CODE HERE!

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
# YOUR CODE HERE!

# Print summary statistics
# YOUR CODE HERE!

In [ ]:
# Set initial investment
# YOUR CODE HERE!

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $60,000
# YOUR CODE HERE!

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 5 years will end within in the range of"
      f" ${ci_lower_five} and ${ci_upper_five}")

### Ten Years Retirement Option

In [ ]:
# Configuring a Monte Carlo simulation to forecast 10 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Running a Monte Carlo simulation to forecast 10 years cumulative returns
# YOUR CODE HERE!

In [ ]:
# Plot simulation outcomes
# YOUR CODE HERE!

In [ ]:
# Plot probability distribution and confidence intervals
# YOUR CODE HERE!

In [ ]:
# Fetch summary statistics from the Monte Carlo simulation results
# YOUR CODE HERE!

# Print summary statistics
# YOUR CODE HERE!

In [ ]:
# Set initial investment
# YOUR CODE HERE!

# Use the lower and upper `95%` confidence intervals to calculate the range of the possible outcomes of our $60,000
# YOUR CODE HERE!

# Print results
print(f"There is a 95% chance that an initial investment of ${initial_investment} in the portfolio"
      f" over the next 10 years will end within in the range of"
      f" ${ci_lower_ten} and ${ci_upper_ten}")